In [132]:
import numpy as np

In [133]:
import numpy as np

In [134]:
def initializeWeights(layer_dims):
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1]) #*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [135]:
test = initializeWeights([4,3,3,1])
print(test)

{'W1': array([[ 0.50207201, -0.67600515,  0.06832904,  0.00073199],
       [-0.30845414, -0.11818861, -0.22877076, -0.32259881],
       [ 0.10836524,  0.17691851,  0.21065637,  0.37227375]]), 'b1': array([[0.],
       [0.],
       [0.]]), 'W2': array([[-0.22709537, -0.55499945, -0.37305829],
       [ 0.76335124,  0.26988408, -1.17227612],
       [ 0.48421261, -0.15269383,  0.67974033]]), 'b2': array([[0.],
       [0.],
       [0.]]), 'W3': array([[ 0.44050304,  0.68788592, -0.23683552]]), 'b3': array([[0.]])}


In [136]:
def relu(Z):
    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    
    cache = Z 
    return A, cache

In [137]:
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache


In [138]:
def linearForward(A, W, b):
    Z = W.dot(A) + b
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)

    return Z, cache

In [139]:
def linearActivationForwardRelu(aPrev, W, b):
    Z, linearCache = linearForward(aPrev, W, b)
    A, activationCache = relu(Z)

    cache = (linearCache, activationCache)
    
    return A, cache

In [140]:
def linearActivationForwardSigmoid(aPrev, W, b):
    Z, linearCache = linearForward(aPrev, W, b)
    A, activationCache = sigmoid(Z)

    cache = (linearCache, activationCache)
    
    return A, cache

In [141]:
def forwardProp(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2
    
    
    for l in range(1, L):
        A_prev = A 
        A, cache = linearActivationForwardRelu(A_prev, parameters['W' + str(l)], parameters['b' + str(l)])
        caches.append(cache)
    
    
    AL, cache = linearActivationForwardSigmoid(A, parameters['W' + str(L)], parameters['b' + str(L)])
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [142]:
def computeCost(AL, Y):
    m = Y.shape[1]
    cost = (-1/m) * (np.dot(Y, np.log(AL).T) + np.dot((1-Y), np.log(1-AL).T))
    np.squeeze(cost)

    return cost

In [143]:
def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True) 
    
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [144]:
def sigmoid_backward(dA, cache):
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [145]:
def linearBackward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1/m) * np.dot(dZ, A_prev.T)
    db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T,dZ)

    return dA_prev, dW, db

In [146]:
def linearActivationBackwardRelu(dA, cache):
    linear_cache, activation_cache = cache
    dZ = relu_backward(dA, activation_cache)
    
    dA_prev, dW, db = linearBackward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [147]:
def linearActivationBackwardSigmoid(dA, cache):
    linear_cache, activation_cache = cache
    dZ = sigmoid_backward(dA, activation_cache)
    
    dA_prev, dW, db = linearBackward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [148]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

one_hot(np.array([0,1,3,4,0,9]))

array([[1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [149]:
def modelBackward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linearActivationBackwardSigmoid(dAL, current_cache)

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linearActivationBackwardRelu(grads["dA" + str(l + 1)], current_cache)
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [150]:
def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

In [151]:
def updateParameters(parameters, grads, learning_rate):
    L = len(parameters) // 2 

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]

    return parameters

In [152]:
def modelFit(X, Y, layerDims, epochs, learning_rate, print_cost=True):
    costs = []
    
    parameters = initializeWeights(layerDims)
    
    # Descente de gradient
    for i in range(0, epochs):
        AL, caches = forwardProp(X, parameters)
        
        cost = computeCost(AL, Y)

        grads = modelBackward(AL, Y, caches)
        
        parameters = updateParameters(parameters, grads, learning_rate)
        if print_cost and i % 100 == 0 or i == epochs - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == epochs:
            costs.append(cost)
    
    return parameters, costs

In [153]:
def predict(X, y, parameters):
    m = X.shape[1]
    n = len(parameters) // 2 
    p = np.zeros((1,m))
    
    probas, caches = forwardProp(X, parameters)

    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0

    print("Accuracy: "  + str(np.sum((p == y)/m)))
        
    return p

In [154]:
import h5py
import numpy as np

In [155]:
def load_data():
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])

    classes = np.array(test_dataset["list_classes"][:])
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [156]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

In [157]:
print(train_x_orig.shape)

(209, 64, 64, 3)


In [158]:
m_train = train_x_orig.shape[0]
num_px = train_x_orig.shape[1]
m_test = test_x_orig.shape[0]

print ("Number of training examples: " + str(m_train))
print ("Number of testing examples: " + str(m_test))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_x_orig shape: " + str(train_x_orig.shape))
print ("train_y shape: " + str(train_y.shape))
print ("test_x_orig shape: " + str(test_x_orig.shape))
print ("test_y shape: " + str(test_y.shape))

Number of training examples: 209
Number of testing examples: 50
Each image is of size: (64, 64, 3)
train_x_orig shape: (209, 64, 64, 3)
train_y shape: (1, 209)
test_x_orig shape: (50, 64, 64, 3)
test_y shape: (1, 50)


In [159]:
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

train_x = train_x_flatten/255.
test_x = test_x_flatten/255.

print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))

train_x's shape: (12288, 209)
test_x's shape: (12288, 50)


In [162]:
param = modelFit(train_x, train_y, [12288, 64, 32, 16, 1], 2500, 0.009)

Cost after iteration 0: 0.7212545152353563
Cost after iteration 100: 0.594611996749886
Cost after iteration 200: 0.5437067042597034
Cost after iteration 300: 0.48470873199103964
Cost after iteration 400: 0.4286362627808889
Cost after iteration 500: 0.3683182218096129
Cost after iteration 600: 0.2895424499085322
Cost after iteration 700: 0.15229210718248334
Cost after iteration 800: 0.09766842240340667
Cost after iteration 900: 0.06417986282988597
Cost after iteration 1000: 0.043013320966717734
Cost after iteration 1100: 0.03064791562317098
Cost after iteration 1200: 0.02173385426735686
Cost after iteration 1300: 0.016599528332941798
Cost after iteration 1400: 0.012499312215914531
Cost after iteration 1500: 0.009780367679394903
Cost after iteration 1600: 0.007912856562713087
Cost after iteration 1700: 0.006569947698800652
Cost after iteration 1800: 0.005504979404175554
Cost after iteration 1900: 0.004707875541822656
Cost after iteration 2000: 0.004093266632146202
Cost after iteration 21

In [163]:
print(predict(test_x, test_y, param[0]))

Accuracy: 0.78
[[1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1.
  1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1.
  1. 0.]]
